In [ ]:
# Drug classification problem 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_drug =pd.read_csv("../input/drug-classification/drug200.csv")
df_drug.head()

In [ ]:
df_drug.shape

In [ ]:
df_drug.describe()

In [ ]:
df_drug.describe(include=object)

In [ ]:
df_drug.duplicated().sum()

In [ ]:
df_drug.isnull().sum()

In [ ]:
100*df_drug['Drug'].value_counts()/len(df_drug)

# Visualization

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
df_drug.head()

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(x=df_drug['Drug'])
plt.title('COUNTPOT OF TYPE OF DRUGS')
plt.xlabel("Drugs type")
plt.grid(True)
plt.show()

Here ,it is clear the count od drugy and drug x are high while drug c and drug b are less in count

In [ ]:
100*df_drug['Drug'].value_counts()/len(df_drug)

In [ ]:
sns.pairplot(df_drug)
plt.show()

From the graph the disctribution of age is not norma, howevel Na_to_K appear to follow normal distribution

In [ ]:
import scipy.stats as st
print(' Na to k',st.shapiro(df_drug['Na_to_K']))
print("Age",st.shapiro(df_drug['Age']))
# from shapiro test it is clear p value is almost zero ,show distribution is not normal

In [ ]:
sns.heatmap(df_drug.corr(),annot=True)
plt.show()

from the heatmap it is clear that that there is not high or moderate correlation between age and Na_to_K

In [ ]:
fig = px.scatter_3d(df_drug, x='Age', y='Na_to_K', z='BP',
              color='Drug')
fig.show()

Graph shows
1. drug b is generally given to people of high bp and age above 50, drg A is preferred for high bp with age less than 40
2. all other drugs except drug b and drug are give to all age person, 
3. the drug c,drug x, drud b , drug a, are preffered for low level of NA to K , drug x for low and normal bp, drug c for normal bp, and drug a and b fro high bp  

In [ ]:
fig = px.scatter_3d(df_drug, x='Age', y='Na_to_K', z='Cholesterol',
              color='Drug')
fig.show()

from the 2 3d graph it is clear that the drug c is preffered for high cholesterol 

In [ ]:
plt.figure(figsize=(8,6))
sns.stripplot(x=df_drug['BP'],y=df_drug['Age'])
plt.show()

the graph make sure that problem of high BP does not belongs to particular category

In [ ]:
plt.figure(figsize=(8,6))
sns.stripplot(x=df_drug['BP'],y=df_drug['Na_to_K'])
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
sns.stripplot(x=df_drug['Cholesterol'],y=df_drug['Na_to_K'])
plt.show()

Graph shows that there is high cholestrol that have link to low Na_to_k

In [ ]:
plt.figure(figsize=(8,6))
sns.boxplot(x=df_drug['Drug'],y=df_drug['Age'],hue =df_drug['Sex'])
plt.show()

From the graph it  is clear that there is huge difference in gender for the drugB, it brings question whether drug is effective for female with age greater 
than 50 but not in male, or the graph might be like this only due to availability of samples in the data

In [ ]:
plt.figure(figsize=(8,6))
sns.violinplot(x=df_drug['Drug'],y=df_drug['Na_to_K'],hue =df_drug['Sex'],split=True)
plt.show()

From the graph we can make sure that the distribution of drugs gender wise along with sodasium to potasium ratio is not uniform for both sex. It lead us to think in different direction that , is the drug acts differently on the femal and male body or it is just an odd observation in small sample of the data.

### Data conversion

In [ ]:
df_drug.head()

In [ ]:
df_drug = pd.get_dummies(df_drug, columns =['BP','Sex', 'Cholesterol'],drop_first=True)

In [ ]:
df_drug.head()

# Modelling

In [ ]:
# applying base algorithm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
from sklearn.metrics import plot_confusion_matrix, plot_roc_curve, confusion_matrix, classification_report, roc_auc_score, cohen_kappa_score,accuracy_score
from sklearn.model_selection import cross_val_score

In [ ]:
X= df_drug.drop("Drug",axis=1)
y = df_drug['Drug']
le = LabelEncoder()
y = le.fit_transform(y)
dt=DecisionTreeClassifier()
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(y)

In [ ]:
dt_model = dt.fit(X_train,y_train)
y_predict = dt_model.predict(X_test)
print(confusion_matrix(y_test,y_predict))
print(classification_report(y_test,y_predict,digits=6))

In [ ]:
roc_auc_score(y_test,dt_model.predict_proba(X_test),multi_class='ovr')



# Important feature

In [ ]:
feat =dt_model.feature_importances_
df_imp_dt =pd.DataFrame(feat,index = df_drug.drop("Drug",axis=1).columns,columns=['val'])
df_imp_dt.sort_values(by='val',ascending=False)

In [ ]:
barplot_feature =px.bar(df_imp_dt.sort_values(by='val',ascending=False))
barplot_feature.show()

# Improvng model with random forest

In [ ]:
rf =RandomForestClassifier()
rf_model =rf.fit(X_train,y_train)
y_predict = rf_model.predict(X_test)
print(confusion_matrix(y_test,y_predict))
print(classification_report(y_test,y_predict,digits=6))

In [ ]:
roc_auc_score(y_test,rf_model.predict_proba(X_test),multi_class='ovo')

In [ ]:
feat =rf_model.feature_importances_
df_imp_rf =pd.DataFrame(feat,index = df_drug.drop("Drug",axis=1).columns,columns=['val'])
df_imp_rf.sort_values(by='val',ascending=False)

barplot_feature_random_forest =px.bar(df_imp_rf.sort_values(by='val',ascending=False))
barplot_feature_random_forest.show()

In [ ]:
plot_confusion_matrix(rf_model,X_test,y_test)
plt.title("CONFUSION MATRIX OF RANDOM FOREST MODEL")
plt.show()  # in the matrix  0,1,2,3,4 refers to ['DrugY' 'drugA' 'drugB' 'drugC' 'drugX'] respcetively

In [ ]:
print("Kappa score")
print(cohen_kappa_score(y_test,y_predict))
print("AUC score")
y_predict_prob = rf_model.predict_proba(X_test)
print(roc_auc_score(y_test,y_predict_prob,multi_class='ovr'))

Since the decision tree is performin well, there is no need to apply approach like random forest , but if the aim is to make the model robust than random
forest model will be great